In [1]:
# import librerie
import os
import tweepy
import facebook
import requests
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import json

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
# configuration file
config = {}
config_path = os.path.join(os.path.abspath('../../'))
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
fb_app_id=config['FB_APP_ID']
fb_app_secret=config['FB_APP_SECRET']
fb_app_userid=config['FB_APP_USERID']

dir_out = os.path.join(os.path.abspath(''),'output')

In [3]:
# get access token
payload = {'grant_type': 'client_credentials', 'client_id': fb_app_id, 'client_secret': fb_app_secret}
response = requests.post('https://graph.facebook.com/oauth/access_token?', params = payload)
json_data = json.loads(response.text)
access_token = json_data['access_token']

In [4]:
# get graph Facebook
graph = facebook.GraphAPI(access_token=access_token, version='2.7')

In [5]:
# Candidati Elezioni
users = [
    {'user':'Renzi','user_id':'113335124914'},
    {'user':'Di Maio','user_id':'522391027797448'},
    {'user':'Salvini','user_id':'252306033154'},
    {'user':'Pisapia','user_id':'112352038802143'},
    {'user':'Meloni','user_id':'38919827644'},
    {'user':'Berlusconi','user_id':'116716651695782'}
]

In [6]:
# get today's date
todays_date = datetime.datetime.now()

In [7]:
# get last post
def get_last_post(fb_user, fb_id, graph, date):
    info = graph.get_object(id=fb_id, fields='posts')
    
    dt = info['posts']['data'][0]['created_time']
    id_post = info['posts']['data'][0]['id']
    try:
        msg = info['posts']['data'][0]['message']
    except:
        msg = info['posts']['data'][1]['message']
    likes = graph.get_connections(id=id_post, connection_name='likes', summary='true')['summary']['total_count']
    
    last_post = {'user':fb_user,'dt_rif':todays_date,'dt_post':dt,'id_post':id_post, 'msg':msg, 'likes':likes}
    
    return last_post

In [8]:
fb_posts = []
for user in users:
    fb_posts.append(get_last_post(user['user'],user['user_id'],graph,todays_date))

In [9]:
df = pd.DataFrame(fb_posts)

In [10]:
df

,dt_post,dt_rif,id_post,likes,msg,user
0,2017-11-26T18:01:23+0000,2017-11-27 18:10:48.769235,113335124914_10155450528999915,5345,È stata una delle edizioni della Leopolda più ...,Renzi
1,2017-11-27T15:02:31+0000,2017-11-27 18:10:48.769235,522391027797448_1532908860078988,982,Da oggi parte la nostra campagna elettorale! N...,Di Maio
2,2017-11-27T16:23:50+0000,2017-11-27 18:10:48.769235,252306033154_10155333732628155,1544,Nella vita privata ognuno è libero di stare co...,Salvini
3,2017-10-18T11:07:33+0000,2017-11-27 18:10:48.769235,112352038802143_1437974499573217,330,Ieri ho incontrato un gruppo di Precari Uniti ...,Pisapia
4,2017-11-27T16:31:01+0000,2017-11-27 18:10:48.769235,38919827644_10155844812207645,750,"Mancano pochi mesi al voto e, magicamente, il ...",Meloni
5,2017-11-27T18:02:37+0000,2017-11-27 18:10:48.769235,116716651695782_1757690284265069,267,Ringrazio ancora una volta tutti coloro che mi...,Berlusconi


In [11]:
# get database connection
db=config['DATABASE_ELE']
schema=config['SCHEMA_ELE']
engine = create_engine(db)

In [12]:
# write to db
df.to_sql('fb_posts', engine, schema=schema, if_exists='append')